In [2]:
#beautiful soup imports
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import time
import random
from time import sleep


# selenium imports
import selenium
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from time import sleep
from selenium.webdriver.common.action_chains import ActionChains

# Airbnb Webscraping
### Imports

Importing a list of locations from the cleaning notebook that will be used to update airbnb links.

In [8]:
url_sel = pd.read_csv("data/cleaned_data_exports/location_for_scraping_selenium_links.csv")

### Resort List

Importing a .csv that I updated in excel to match Airbnb's URL naming convention. I plan to scrape information on Airbnb costs for 2 and 4 guest airbnbs from December - May.

In [76]:
#updating names to properly input into url
url_sel['city'] = url_sel['city'].replace(" ", "-", regex=True)
url_sel['state'] = url_sel['state'].replace("%20", "-", regex=True)

### Creating list of airbnb URL's for each resort
This will be used to scrape 28 airbnb prices per resort for 2 guest Airbnb's

In [80]:
#making list out of strings that will be added to url
city_list = url_sel['city'].tolist()
state_list = url_sel['state'].tolist()
city_url = url_sel['city_2'].tolist()
state_url = url_sel['state_2'].tolist()

#airbnb nightly cost for four guests in december
sel_list = [f"https://www.airbnb.com/s/{city}--{state}--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2023-06-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&query={state_2}&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&adults=4&source=structured_search_input_header&search_type=autocomplete_click"
            for city, state, state_2 in zip(city_list, state_list, state_url)]

In [81]:
#checking link
sel_list[2]

'https://www.airbnb.com/s/Snoquale-Pass--Washington--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2023-06-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&query=Snoquale%20Pass%2C%20WA&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&adults=4&source=structured_search_input_header&search_type=autocomplete_click'

In [82]:
#checking if links are active
for link in sel_list:
    if check_link_status(link):
        print(f"{link} is active.")
    else:
        print(f"{link} is not active or cannot be reached.")

https://www.airbnb.com/s/Chewelah--Washington--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2023-06-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&query=Chewelah%2C%20WA&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&adults=4&source=structured_search_input_header&search_type=autocomplete_click is active.
https://www.airbnb.com/s/Hastings--Minnesota--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2023-06-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&query=Hastings%2C%20MN&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&adults=4&source=structured_search_input_header&search_type=autocomplete_click is active.
https://www.airbnb.com/s/Snoquale-Pass--Washington--United-States/homes?tab_id=home_tab&refinement_paths%5B%5

https://www.airbnb.com/s/Vail--Colorado--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2023-06-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&query=Vail%2C%20CO&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&adults=4&source=structured_search_input_header&search_type=autocomplete_click is active.
https://www.airbnb.com/s/Logan--Utah--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2023-06-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&query=Logan%2C%20UT&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&adults=4&source=structured_search_input_header&search_type=autocomplete_click is active.
https://www.airbnb.com/s/Banner--Elk--North-Carolina--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible

https://www.airbnb.com/s/Bolton--Vermont--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2023-06-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&query=Bolton%2C%20VT&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&adults=4&source=structured_search_input_header&search_type=autocomplete_click is active.
https://www.airbnb.com/s/Truckee--California--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2023-06-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&query=Truckee%2C%20CA&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&adults=4&source=structured_search_input_header&search_type=autocomplete_click is active.
https://www.airbnb.com/s/Peninsula--Ohio--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexibl

https://www.airbnb.com/s/Orchard-Park--New-York--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2023-06-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&query=Orchard%20Park%2C%20NY&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&adults=4&source=structured_search_input_header&search_type=autocomplete_click is active.
https://www.airbnb.com/s/East-Burke--Vermont--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2023-06-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&query=East%20Burke%2C%20VT&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&adults=4&source=structured_search_input_header&search_type=autocomplete_click is active.
https://www.airbnb.com/s/Cadill--Michigan--United-States/homes?tab_id=home_tab&refinement_paths%5

https://www.airbnb.com/s/Copper-Mountain--Colorado--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2023-06-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&query=Copper%20Mountain%2C%20CO&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&adults=4&source=structured_search_input_header&search_type=autocomplete_click is active.
https://www.airbnb.com/s/North-Conway--New-Hampshire--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2023-06-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&query=North%20Conway%2C%20NH&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&adults=4&source=structured_search_input_header&search_type=autocomplete_click is active.
https://www.airbnb.com/s/Mt.-Crested-Butte--Colorado--United-States/homes?tab_id=

...

https://www.airbnb.com/s/Big-Bear-Lake--California--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2023-06-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&query=Big%20Bear%20Lake%2C%20CA&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&adults=4&source=structured_search_input_header&search_type=autocomplete_click is active.
https://www.airbnb.com/s/Mansfield--Ohio--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2023-06-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&query=Mansfield%2C%20OH&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&adults=4&source=structured_search_input_header&search_type=autocomplete_click is active.
https://www.airbnb.com/s/Running-Springs--California--United-States/homes?tab_id=home_tab&refineme

https://www.airbnb.com/s/Norden--California--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2023-06-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&query=Norden%2C%20CA&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&adults=4&source=structured_search_input_header&search_type=autocomplete_click is active.
https://www.airbnb.com/s/Banner-Elk--North-Carolina--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2023-06-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&query=Banner%20Elk%2C%20NO&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&adults=4&source=structured_search_input_header&search_type=autocomplete_click is active.
https://www.airbnb.com/s/Warren--Vermont--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%

https://www.airbnb.com/s/Choteau--Montana--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2023-06-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&query=Choteau%2C%20MO&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&adults=4&source=structured_search_input_header&search_type=autocomplete_click is active.
https://www.airbnb.com/s/Harbor-Springs--Michigan--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2023-06-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&query=Harbor%20Springs%2C%20MI&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&adults=4&source=structured_search_input_header&search_type=autocomplete_click is active.
https://www.airbnb.com/s/Glen-Arbor--Michigan--United-States/homes?tab_id=home_tab&refinement_paths%5

https://www.airbnb.com/s/White-Pass--Washington--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2023-06-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&query=White%20Pass%2C%20WA&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&adults=4&source=structured_search_input_header&search_type=autocomplete_click is active.
https://www.airbnb.com/s/Pinedale--Wyoming--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2023-06-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&query=Pinedale%2C%20WY&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&adults=4&source=structured_search_input_header&search_type=autocomplete_click is active.
https://www.airbnb.com/s/Upson--Wisconsin--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2F

https://www.airbnb.com/s/Woodbury--Connecticut--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2023-06-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&query=Woodbury%2C%20CO&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&adults=4&source=structured_search_input_header&search_type=autocomplete_click is active.
https://www.airbnb.com/s/Westernville--New-York--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2023-06-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&query=Westernville%2C%20NE&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=december&adults=4&source=structured_search_input_header&search_type=autocomplete_click is active.
https://www.airbnb.com/s/Slocum--Rhode-Island--United-States/homes?tab_id=home_tab&refinement_paths%5

In [83]:
#saving final url list as new csv
#url_sel['url'] = sel_list
#url_sel.to_csv('cleaned_data_exports/airbnb_url_list.csv')

## Testing Webscape
Below I am creating a list of five resorts that I will use to create a function to pull airbnb prices.

In [9]:
#importing saved file from scrape above
final_url_df = pd.read_csv("data/cleaned_data_exports/airbnb_url_list.csv")

#dropping unnamed columnd
final_url_df.drop(columns="Unnamed: 0", inplace=True)

In [12]:
final_url_df.head()

,ski_resort,city,city_2,state,abbreviation,state_2,url,url_2
0,49 Degrees North,Chewelah,Chewelah%2C%20,Washington,WA,Chewelah%2C%20WA,https://www.airbnb.com/s/Chewelah--Washington-...,https://www.airbnb.com/s/Chewelah--Washington-...
1,Afton Alps,Hastings,Hastings%2C%20,Minnesota,MN,Hastings%2C%20MN,https://www.airbnb.com/s/Hastings--Minnesota--...,https://www.airbnb.com/s/Hastings--Minnesota--...
2,Alpental,Snoquale-Pass,Snoquale%20Pass%2C%20,Washington,WA,Snoquale%20Pass%2C%20WA,https://www.airbnb.com/s/Snoquale-Pass--Washin...,https://www.airbnb.com/s/Snoquale-Pass--Washin...
3,Alpine Valley Ohio,Chesterland,Chesterland%2C%20,Ohio,OH,Chesterland%2C%20OH,https://www.airbnb.com/s/Chesterland--Ohio--Un...,https://www.airbnb.com/s/Chesterland--Ohio--Un...
4,Alpine Valley Wisconsin,East-Troy,East%20Troy%2C%20,Wisconsin,WI,East%20Troy%2C%20WI,https://www.airbnb.com/s/East-Troy--Wisconsin-...,https://www.airbnb.com/s/East-Troy--Wisconsin-...


In [13]:
#testing five results

#these are urls from the first page of airbnb results
fin_url_list_1 = final_url_df['url'].head()

#these are urls from the second page of airbnb results
fin_url_list_2 = final_url_df['url_2'].head()

In [14]:
# Loop through the URLs from both lists simultaneously
for url1, url2 in zip(final_url_df['url'].head(), final_url_df['url_2'].head()):
    # get request to the URL in url_list1
    response1 = requests.get(url1)
    soup1 = BeautifulSoup(response1.text, 'html.parser')
    
    # get request to the URL in url_list2
    response2 = requests.get(url2)
    soup2 = BeautifulSoup(response2.text, 'html.parser')

    # finding elements from the first URL
    elements1 = soup1.find_all('span', class_='_tyxjp1')
    for element1 in elements1:
        print("URL 1:", element1.text)

    # inding elements from the second URL
    elements2 = soup2.find_all('span', class_='_tyxjp1')
    for element2 in elements2:
        print("URL 2:", element2.text)

URL 1: $204 
URL 1: $166 
URL 1: $114 
URL 1: $172 
URL 1: $409 
URL 1: $208 
URL 1: $247 
URL 1: $435 
URL 1: $119 
URL 1: $89 
URL 1: $191 
URL 1: $87 
URL 1: $330 
URL 1: $130 
URL 1: $178 
URL 1: $79 
URL 1: $292 
URL 1: $200 
URL 2: $157 
URL 2: $184 
URL 2: $100 
URL 2: $302 
URL 2: $106 
URL 2: $148 
URL 2: $250 
URL 2: $225 
URL 2: $129 
URL 2: $359 
URL 2: $141 
URL 2: $288 
URL 2: $125 
URL 2: $225 
URL 2: $250 
URL 2: $270 
URL 2: $127 
URL 1: $389 
URL 1: $243 
URL 1: $811 
URL 1: $204 
URL 1: $210 
URL 1: $209 
URL 1: $175 
URL 1: $151 
URL 1: $700 
URL 1: $247 
URL 1: $265 
URL 1: $225 
URL 1: $400 
URL 1: $117 
URL 2: $700 
URL 2: $151 
URL 2: $169 
URL 2: $117 
URL 2: $419 
URL 2: $308 
URL 2: $470 
URL 2: $158 
URL 2: $177 
URL 2: $191 
URL 2: $159 
URL 2: $310 
URL 2: $175 
URL 2: $209 
URL 2: $330 
URL 2: $235 
URL 2: $191 
URL 1: $280 
URL 1: $233 
URL 1: $1,271 
URL 1: $411 
URL 1: $134 
URL 1: $174 
URL 1: $305 
URL 1: $613 
URL 1: $311 
URL 1: $368 
URL 1: $588 


Loop to pull out airbnb price from the first two pages for each city.

In [426]:
result_dict_2 = {}

# Loop through the URLs from both lists at the same time
for url1, url2, ski_resort in zip(final_url_df['url'], final_url_df['url_2'], final_url_df['ski_resort']):
    # Make a reguest to first list
    response1 = requests.get(url1)
    soup1 = BeautifulSoup(response1.text, 'html.parser')

    # Make a reguest to second list
    response2 = requests.get(url2)
    soup2 = BeautifulSoup(response2.text, 'html.parser')

    # finding elements
    elements1 = soup1.find_all('span', class_='_tyxjp1')
    result_dict_2.setdefault(ski_resort, []).extend([element1.text.strip() for element1 in elements1])

    # finding elements
    elements2 = soup2.find_all('span', class_='_tyxjp1')
    result_dict_2.setdefault(ski_resort, []).extend([element2.text.strip() for element2 in elements2])

    #sleeping
    sleep_duration = random.randint(4, 10)
    time.sleep(sleep_duration)

In [430]:
#saving results as dataframe
for x in result_dict_2.keys():
    december_df_2[x] = pd.DataFrame(result_dict_2[x], columns=[x])

In [431]:
#saving to csv
#december_df_2.to_csv("cleaned_data_exports/december_soup_2.csv")

In [432]:
#replacing dollar signs
december_df_2 = december_df_2.replace(to_replace=r'[^0-9.-]', value='', regex=True)

#converting dataype
december_df_2 = december_df_2.astype(float)

#transposing dataframe
december_df_2 = december_df_2.describe().T

#saving index as new
december_df_2["ski_resort"] = december_df_2.index.copy()

#resetting index
december_df_2 = december_df_2.reset_index(drop=True)

In [433]:
december_df_2.sort_values(by='mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max,ski_resort
208,35.0,558.542857,303.838903,201.0,340.50,514.0,685.00,1650.0,Sundance
200,32.0,439.218750,198.639361,207.0,274.00,403.0,504.75,990.0,Stratton Mountain
61,35.0,436.314286,145.816115,226.0,299.50,438.0,568.00,658.0,Jackson Hole
218,35.0,425.457143,132.685241,229.0,318.50,424.0,497.00,711.0,Telluride
233,35.0,421.485714,172.170123,154.0,292.00,378.0,477.00,900.0,Vail
...,...,...,...,...,...,...,...,...,...
21,30.0,151.500000,78.881294,69.0,94.00,127.0,186.75,440.0,Devils Head
220,35.0,150.542857,57.097617,75.0,100.00,150.0,182.00,300.0,Teton Pass
82,34.0,145.558824,47.073889,65.0,113.25,145.0,165.00,250.0,Magic Mountain Idaho
151,5.0,130.800000,73.754322,56.0,58.00,155.0,156.00,229.0,Ragged Mountain


In [434]:
final_dec_airbnb_df = pd.concat([december_df, december_df_2])

In [436]:
#final_dec_airbnb_df.to_csv("cleaned_data_exports/dec_airbnb_mean_final.csv")

In [438]:
final_dec_airbnb_df.sort_values(by="mean", ascending=False)

,count,mean,std,min,25%,50%,75%,max,ski_resort
15,35.0,624.257143,318.057000,216.0,422.50,577.0,697.50,1402.0,Aspen Snowmass
208,35.0,558.542857,303.838903,201.0,340.50,514.0,685.00,1650.0,Sundance
200,32.0,439.218750,198.639361,207.0,274.00,403.0,504.75,990.0,Stratton Mountain
61,35.0,436.314286,145.816115,226.0,299.50,438.0,568.00,658.0,Jackson Hole
218,35.0,425.457143,132.685241,229.0,318.50,424.0,497.00,711.0,Telluride
...,...,...,...,...,...,...,...,...,...
21,30.0,151.500000,78.881294,69.0,94.00,127.0,186.75,440.0,Devils Head
220,35.0,150.542857,57.097617,75.0,100.00,150.0,182.00,300.0,Teton Pass
82,34.0,145.558824,47.073889,65.0,113.25,145.0,165.00,250.0,Magic Mountain Idaho
151,5.0,130.800000,73.754322,56.0,58.00,155.0,156.00,229.0,Ragged Mountain


### Function

Making a function that combines the code above so you can loop through different months and pull prices from the first two airbnb pages.

In [589]:
def scrape_airbnb_prices(month, adults):
    
    #making a copy of the final url
    month_url_df = final_url_df.copy()
    #in the .csv file the url link includes "december" and "4 adults" so we will be updating this for each loop past on the params
    month_url_df = month_url_df.replace("december", month, regex=True)
    month_url_df = month_url_df.replace("adults=4", f"adults={adults}", regex=True)
    
    month_dict_1 = {}

    # Loop through the URLs from both lists at the same time
    for url1, url2, ski_resort in zip(month_url_df['url'], month_url_df['url_2'], month_url_df['ski_resort']):
        # Make a request to the first url list
        response1 = requests.get(url1)
        soup1 = BeautifulSoup(response1.text, 'html.parser')

        # Make a request to the second url list
        response2 = requests.get(url2)
        soup2 = BeautifulSoup(response2.text, 'html.parser')

        # finding elements
        elements1 = soup1.find_all('span', class_='_tyxjp1')
        month_dict_1.setdefault(ski_resort, []).extend([element1.text.strip() for element1 in elements1])

        #finding elements
        elements2 = soup2.find_all('span', class_='_tyxjp1')
        month_dict_1.setdefault(ski_resort, []).extend([element2.text.strip() for element2 in elements2])

        #sleeping
        sleep_duration = random.randint(4, 10)
        time.sleep(sleep_duration) 

    for x in month_dict_1.keys():
        month_df = pd.DataFrame(month_dict_1[x], columns=[x])

    for x in month_dict_1.keys():
        month_df[x] = pd.DataFrame(month_dict_1[x], columns=[x])
    
    #saving original soup results
    #month_df.to_csv(f"cleaned_data_exports/test/{month}_{adults}_soup.csv")

    #replacing dollar signs
    month_df = month_df.replace(to_replace=r'[^0-9.-]', value='', regex=True)

    #converting dataype
    month_df = month_df.astype(float)

    #transposing dataframe
    month_df = month_df.describe().T

    #saving index as new
    month_df["ski_resort"] = month_df.index.copy()

    #resetting index
    month_df = month_df.reset_index(drop=True)
    
    #month_df.to_csv(f"cleaned_data_exports/test/{month}_{adults}_airbnb_mean_final.csv")
    
    return(month_df.sort_values(by="mean", ascending=False))

## Running final scrape from December to May.

### Scraping 2 Guest Nightly Prices

In [590]:
month_list = ["december", "january", "february", "march", "april", "may"]

for x in month_list:
    scrape_airbnb_prices(x, 2)
    
    #adding sleep time
    time.sleep(120)

### Scraping 4 Guest Nightly Prices

In [ ]:
month_list = ["december", "january", "february", "march", "april", "may"]

for x in month_list:
    scrape_airbnb_prices(x, 4)
    
    #adding sleep time
    time.sleep(120)